In [1]:
#Importação das bibliotecas necessárias
import pandas as pd
import glob
import datetime

In [2]:
#Atribuindo a hora atual para variável a fim de avaliar ao final o tempo de execução
hora_inicio = datetime.datetime.now()

In [3]:
# Função para tratar informações de data dentro do SPED
data = ''
def FormatarData(variavel, num):
    global data
    data = variavel[num][:2] + '/' + variavel[num][2:4] + '/' + variavel[num][4:]
    return data

In [4]:
#Lista para armazenar as informações de todos os arquivos lidos
listaArquivos = []
#Lista para armazenar as informações de linha de cada arquivo
listaLinhas = []

In [5]:
#Usando o glob para ler todos os arquivos txt dentro da pasta referenciada
path_efd = glob.glob('.\data\*.txt')

#Mostrar todos os arquivos encontrados
#print(path_efd)

In [6]:
for i in path_efd:
    listaLinhas = []
    dictCabecalho = {}
    dictLinha = {}

    df = open(i,'r', encoding="Latin-1")

    for line in df:
        if '|0000|' in line:
            arq = line.split("|")
            dictCabecalho['TP_ESCRITURACAO'] = arq[3]
            dictCabecalho['NUM_RECIBO'] = arq[5]
            FormatarData(arq, 6)
            dictCabecalho['DT_INI'] = data
            FormatarData(arq, 7)
            dictCabecalho['DT_FIN'] = data
            dictCabecalho['EMPRESA'] = arq[8]
            dictCabecalho['CNPJ'] = arq[9]

        elif '|C100|' in line:
            linha = line.split('|')

            dictLinha['REG'] = linha[1]
            dictLinha['IND_OPER'] = linha[2]
            dictLinha['IND_EMIT'] = linha[3]
            dictLinha['COD_PART'] = linha[4]
            dictLinha['COD_MOD'] = linha[5]
            dictLinha['COD_SIT'] = linha[6]
            dictLinha['SER'] = linha[7]
            dictLinha['NUM_DOC'] = linha[8]
            dictLinha['CHV_NFE'] = linha[9]
            FormatarData(linha,10)
            dictLinha['DT_DOC'] = data
            FormatarData(linha,11)
            dictLinha['DT_E_S'] = data
            dictLinha['VL_DOC'] = linha[12].replace('.','').replace(',','.')
            dictLinha['IND_PGTO'] = linha[13]
            dictLinha['VL_DESC'] = linha[14]
            dictLinha['VL_ABAT_NT'] = linha[15]
            dictLinha['VL_MERC'] = linha[16].replace('.','').replace(',','.')
            dictLinha['IND_FRT'] = linha[17]
            dictLinha['VL_FRT'] = linha[18]
            dictLinha['VL_SEG'] = linha[19]
            dictLinha['VL_OUT_DA'] = linha[20]
            dictLinha['VL_BC_ICMS'] = linha[21].replace('.','').replace(',','.')
            dictLinha['VL_ICMS'] = linha[22].replace('.','').replace(',','.')
            dictLinha['VL_BC_ICMS_ST'] = linha[23]
            dictLinha['VL_ICMS_ST'] = linha[24]
            dictLinha['VL_IPI'] = linha[25]
            dictLinha['VL_PIS'] = linha[26]
            dictLinha['VL_COFINS'] = linha[27]
            dictLinha['VL_PIS_ST'] = linha[28]
            dictLinha['VL_COFINS_ST'] = linha[29]            
            

            listaLinhas.append(dictLinha)
            dictLinha = {}

        elif '|9001|' in line:
            break

    listaArquivos.append({'cabecalho': dictCabecalho, 'linhas': listaLinhas})

In [7]:
#Utilização do normalize para alinhar as linhas (Bloco |C100|) com os cabeçalhos (Bloco |0000|)
df = pd.json_normalize(listaArquivos, errors='ignore',record_path= 'linhas',
meta=[
['cabecalho','TP_ESCRITURACAO'],
['cabecalho','NUM_RECIBO'],
['cabecalho','DT_INI'],
['cabecalho','DT_FIN'],
['cabecalho','EMPRESA'],
['cabecalho','CNPJ']])

In [8]:
#Argumento para o pandas mostrar sempre todas as colunas do dataframe
pd.set_option('display.max_columns', None)

In [9]:
#Retorna as informações de todo o dataframe gerado
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   REG                        9 non-null      object
 1   IND_OPER                   9 non-null      object
 2   IND_EMIT                   9 non-null      object
 3   COD_PART                   9 non-null      object
 4   COD_MOD                    9 non-null      object
 5   COD_SIT                    9 non-null      object
 6   SER                        9 non-null      object
 7   NUM_DOC                    9 non-null      object
 8   CHV_NFE                    9 non-null      object
 9   DT_DOC                     9 non-null      object
 10  DT_E_S                     9 non-null      object
 11  VL_DOC                     9 non-null      object
 12  IND_PGTO                   9 non-null      object
 13  VL_DESC                    9 non-null      object
 14  VL_ABAT_NT    

In [10]:
#Alterando o dtype das colunas selecionadas para númerico (float)
df[["VL_MERC","VL_BC_ICMS","VL_ICMS"]] = df[["VL_MERC","VL_BC_ICMS","VL_ICMS"]].apply(pd.to_numeric)

In [11]:
#Criar nova coluna com o valor da nova base de cálculo para PIS e COFINS
df['NOVA_BC_PISCOFINS'] = df['VL_BC_ICMS'] - df['VL_ICMS']

In [12]:
#Variável para armazenar a alíquota de PIS e COFINS somadas para o regime não cumulativo
aliquota = 0.0925

#Criar nova coluna com o novo valor a ser pago de PIS E COFINS
df['NOVO_VALOR_PISCOFINS'] = (df['NOVA_BC_PISCOFINS'] * aliquota).round(decimals=2)

In [13]:
#Mostrar o dataframe gerado
#df

In [14]:
#Salvar o dataframe em csv
df.to_csv(r'.\output\result.csv', sep=';', encoding='utf-8', index=False)

In [15]:
print('Iniciou em {}'.format(hora_inicio))
print('Terminou em {}'.format(datetime.datetime.now()))

Iniciou em 2021-05-24 21:53:17.329382
Terminou em 2021-05-24 21:53:17.581584
